<a href="https://colab.research.google.com/github/JasperLS/UFC_Fight_Predictions/blob/master/scraping/ufc_stats_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrape Data


### General Prep

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time as time_lib
import os
import csv
import pandas as pd
import sys
chrome_options = Options()
chrome_options.add_argument("--headless")
import datetime
sys.path.insert(1, os.getcwd().replace('scraping','')) # allows config import from one level above
import config as cfg

browser = webdriver.Chrome(cfg.path_chromedriver, options=chrome_options)
date =str(datetime.datetime.today().strftime('%Y-%m-%d'))
print(date)

2020-11-27


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"#"last_expr" #all for debugging better

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

import warnings
warnings.filterwarnings("ignore")

### Get All Stats

In [3]:
# define input
alphabet = 'A B C D E F G H I J K L M N O P Q R S T U V W X Y Z'.split(' ')
FID = 0

cols_history = ['Result','Fighter_1','Fighter_2','STR_1','STR_2','TD_1','TD_2','SUB_1','SUB_2','PASS_1','PASS_2','Event','Date','Method','Round','Time']
cols_match = ['MID','FID','Result_F1','Event','Date','Method','Round','Time',                                         # before click
              'Type','T_Format','Ref','Details',                                                             # after click
              'Fighter_1','Fighter_2',
              'KD_1','KD_2','SigStr_e_1','SigStr_t_1','SigStr_e_2','SigStr_t_2','TotStr_e_1','TotStr_t_1','TotStr_e_2','TotStr_t_2','TD_e_1','TD_t_1','TD_e_2','TD_t_2',
              'SUB_1','SUB_2','PASS_1','PASS_2','REV_1','REV_2',
              'Head_e_1','Head_t_1','Head_e_2','Head_t_2','Body_e_1','Body_t_1','Body_e_2','Body_t_2','Leg_e_1','Leg_t_1','Leg_e_2','Leg_t_2',
              'Distance_e_1','Distance_t_1','Distance_e_2','Distance_t_2','Clinch_e_1','Clinch_t_1','Clinch_e_2','Clinch_t_2','Ground_e_1','Ground_t_1','Ground_e_2','Ground_t_2']
cols_plan = ['FID','Fighter_1','Fighter_2']
cols_all_fighters = ['FID','First','Last','Nick','HT','WT','Reach','Stance','W','L','D']
cols_profile = ['FID','HT','WT','Reach','Stance','DOB']

plan = pd.DataFrame(columns = cols_plan)
plan_path = "".join([cfg.path_data_output,date,'_Plan_List.csv'])
plan.to_csv(plan_path, mode = 'w',header = True, index=False)

matches = pd.DataFrame(columns = cols_match)
matches_path = "".join([cfg.path_data_output,date,'_Matches_List.csv'])
matches.to_csv(matches_path, mode = 'w',header = True, index=False)

fighters = pd.DataFrame(columns = cols_all_fighters)
fighters_path = "".join([cfg.path_data_output,date,'_Fighters_List.csv'])
fighters.to_csv(fighters_path, mode = 'w',header = True,index=False)

profile = pd.DataFrame(columns = cols_profile)
profile_path = "".join([cfg.path_data_output,date,'_Profiles_List.csv'])
profile.to_csv(profile_path, mode = 'w',header = True,index=False)

In [4]:
def match_page(browser,Event,Date,Method,Roundn,Time,FID,cols_match):
    MID = browser.current_url.split('/')[-1]
    Result = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[1]/i').text
    Type = browser.find_element_by_xpath(f'/html/body/section/div/div/div[2]/div[1]/i').text
    T_Format = browser.find_element_by_xpath(f'/html/body/section/div/div/div[2]/div[2]/p[1]/i[4]').text.replace('TIME FORMAT: ','')
    try: Ref = browser.find_element_by_xpath(f'/html/body/section/div/div/div[2]/div[2]/p[1]/i[5]/span').text
    except: Ref = None
    Details = browser.find_element_by_xpath(f'/html/body/section/div/div/div[2]/div[2]/p[2]').text.replace('DETAILS: ','')
    
    try:    Fighter_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[1]/div/h3/a').text
    except: Fighter_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[1]/div/h3/span').text
    try:    Fighter_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[2]/div/h3/a').text
    except: Fighter_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[2]/div/h3/span').text
    
    try: # get detailed significant strike infos
        KD_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[2]/p[1]').text
        KD_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[2]/p[2]').text

        SigStr_e_1,SigStr_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[3]/p[1]').text.split(' of ')
        SigStr_e_2,SigStr_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[3]/p[2]').text.split(' of ')

        TotStr_e_1,TotStr_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[5]/p[1]').text.split(' of ')
        TotStr_e_2,TotStr_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[5]/p[2]').text.split(' of ')

        TD_e_1,TD_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[6]/p[1]').text.split(' of ')
        TD_e_2,TD_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[6]/p[2]').text.split(' of ')

        SUB_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[8]/p[1]').text
        SUB_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[8]/p[2]').text
        PASS_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[9]/p[1]').text
        PASS_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[9]/p[2]').text
        REV_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[10]/p[1]').text
        REV_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[10]/p[2]').text

        Head_e_1,Head_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[4]/p[1]').text.split(' of ')
        Head_e_2,Head_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[4]/p[2]').text.split(' of ')

        Body_e_1,Body_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[5]/p[1]').text.split(' of ')
        Body_e_2,Body_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[5]/p[2]').text.split(' of ')

        Leg_e_1,Leg_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[6]/p[1]').text.split(' of ')
        Leg_e_2,Leg_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[6]/p[2]').text.split(' of ')

        Distance_e_1,Distance_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[7]/p[1]').text.split(' of ')
        Distance_e_2,Distance_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[7]/p[2]').text.split(' of ')

        Clinch_e_1,Clinch_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[8]/p[1]').text.split(' of ')
        Clinch_e_2,Clinch_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[8]/p[2]').text.split(' of ')

        Ground_e_1,Ground_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[9]/p[1]').text.split(' of ')
        Ground_e_2,Ground_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[9]/p[2]').text.split(' of ')

    except:
        KD_1,KD_2,SigStr_e_1,SigStr_t_1,SigStr_e_2,SigStr_t_2,TotStr_e_1,TotStr_t_1,TotStr_e_2,TotStr_t_2,TD_e_1,TD_t_1,TD_e_2,TD_t_2,\
                                      SUB_1,SUB_2,PASS_1,PASS_2,REV_1,REV_2,\
                                      Head_e_1,Head_t_1,Head_e_2,Head_t_2,Body_e_1,Body_t_1,Body_e_2,Body_t_2,Leg_e_1,Leg_t_1,Leg_e_2,Leg_t_2,\
                                      Distance_e_1,Distance_t_1,Distance_e_2,Distance_t_2,\
                                      Clinch_e_1,Clinch_t_1,Clinch_e_2,Clinch_t_2,Ground_e_1,Ground_t_1,Ground_e_2,Ground_t_2 = [None]*44
        pass
    
    return pd.DataFrame([[MID,FID,Result,Event,Date,Method,Roundn,Time,
                                          Type,T_Format,Ref,Details,
                                          Fighter_1,Fighter_2,
                                          KD_1,KD_2,SigStr_e_1,SigStr_t_1,SigStr_e_2,SigStr_t_2,TotStr_e_1,TotStr_t_1,TotStr_e_2,TotStr_t_2,TD_e_1,TD_t_1,TD_e_2,TD_t_2,
                                          SUB_1,SUB_2,PASS_1,PASS_2,REV_1,REV_2,
                                          Head_e_1,Head_t_1,Head_e_2,Head_t_2,Body_e_1,Body_t_1,Body_e_2,Body_t_2,Leg_e_1,Leg_t_1,Leg_e_2,Leg_t_2,
                                          Distance_e_1,Distance_t_1,Distance_e_2,Distance_t_2,
                                          Clinch_e_1,Clinch_t_1,Clinch_e_2,Clinch_t_2,Ground_e_1,Ground_t_1,Ground_e_2,Ground_t_2]], columns = cols_match)   

In [5]:
def fighter_page(browser,j):
    Event = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[7]/p[1]/a').text
    Date = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[7]/p[2]').text
    Method = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[8]/p[1]').text
    Roundn = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[9]/p').text
    Time = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[10]/p').text
    
    return Event,Date,Method,Roundn,Time

In [6]:
def fighter_profile(browser,FID,cols_profile):
    height = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/ul/li[1]').text.replace('HEIGHT: ','')
    weight = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/ul/li[2]').text.replace('WEIGHT: ','')
    reach = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/ul/li[3]').text.replace('REACH: ','')
    stance = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/ul/li[4]').text.replace('STANCE: ','')
    dob = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/ul/li[5]').text.replace('DOB: ','')
    
    profile = pd.DataFrame([[int(FID),height,weight,reach,stance,dob]],columns = cols_profile)
    return profile

In [7]:
def plan_page(browser, FID,cols_plan):
    try:
        Fighter_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[1]/div/h3/a').text
        Fighter_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[2]/div/h3/a').text
    except:
        Fighter_1,Fighter_2 = 'Error','Error'
        
    return pd.DataFrame([[FID,Fighter_1,Fighter_2]], columns = cols_plan) 

In [8]:
def all_fighters_page(browser,i, FID, cols_all_fighters):
            First = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[1]/a').text
            Last = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[2]/a').text
            Nick = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[3]/a').text
            HT = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[4]').text
            WT = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[5]').text
            Reach = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[6]').text
            Stance = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[7]').text
            W = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[8]').text
            L = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[9]').text
            D = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[10]').text

            fighters = pd.DataFrame([[FID,First, Last, Nick, HT, WT, Reach, Stance, W, L, D]],columns = cols_all_fighters)

            return fighters

In [9]:
# total loop with all infor included
from itertools import count
import random
time_lib.ctime()
FID
for letter in alphabet: # loop through starting letters A to Z
    website = f'http://ufcstats.com/statistics/fighters?char={letter.lower()}&page=all'
    browser.get(website)
    tries = 0
    i=1
    while(True): # loop through list of fighters on page with starting letter
        i = i+1
        try: # get fighter stats
            print('Found fighter')
            fighters = all_fighters_page(browser,i,FID,cols_all_fighters)
            fighters.to_csv(fighters_path, mode = 'a+',header = False, index = False)
        except: pass
        
        try:
            browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[1]/a').click() #Clicking the (i-1)th fighter
            try: 
                profile = fighter_profile(browser,FID,cols_profile)
                profile.to_csv(profile_path, mode = 'a+', header = False, index = False)
            except: pass
            
            for j in count(2): # loop through list of fights per fighter
                try:
                    if browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[1]/p/a/i/i').text == 'NEXT':
                        browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[1]/p/a/i/i').click()#;time_lib.sleep(3) # go to plan page
                        plan = plan_page(browser, FID,cols_plan)
                        plan.to_csv(plan_path, mode = 'a+',header = False, index = False)
                        browser.execute_script("window.history.go(-1)");time_lib.sleep(cfg.sleep+random.random()/100) # go back to fighter page
                    
                    else:
                        Event,Date,Method,Roundn,Time = fighter_page(browser, j)

                        browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[1]/p/a/i/i').click()#;time_lib.sleep(3) # go to match page
                        matches = match_page(browser,Event,Date,Method,Roundn,Time,FID,cols_match)
                        matches.to_csv(matches_path, mode = 'a+',header = False, index = False)
                        browser.execute_script("window.history.go(-1)");time_lib.sleep(cfg.sleep+random.random()/100) # go back to fighter page
                
                except: # end loop trough list of fights
                    print(FID, 'completed with j:',j)
                    FID += 1
                    tries = 0
                    browser.execute_script("window.history.go(-1)");time_lib.sleep(cfg.sleep+random.random()/100) # go back to fighter list page            
                    break
                    
        except: # end loop through list of fighers per starting letter
            if tries <5:
                tries +=1
                i -= 1
                time_lib.sleep(3+random.random()/100)
                website = f'http://ufcstats.com/statistics/fighters?char={letter.lower()}&page=all'
                browser.get(website)
                print("Error reading fighter ", i, ", try: ", tries)
                continue
            print(letter,'max i: ',i,' and time:',time_lib.ctime())
            break

'Thu Sep 24 14:44:23 2020'

0

Found fighter
0 completed with j: 4
Found fighter
1 completed with j: 4
Found fighter
2 completed with j: 23
Found fighter
3 completed with j: 10
Found fighter
4 completed with j: 4
Found fighter
5 completed with j: 5
Found fighter
6 completed with j: 6
Found fighter
7 completed with j: 4
Found fighter
8 completed with j: 6
Found fighter
9 completed with j: 6
Found fighter
10 completed with j: 3
Found fighter
11 completed with j: 6
Found fighter
12 completed with j: 2
Found fighter
13 completed with j: 3
Found fighter
14 completed with j: 11
Found fighter
15 completed with j: 6
Found fighter
16 completed with j: 3
Found fighter
17 completed with j: 4
Found fighter
18 completed with j: 3
Found fighter
19 completed with j: 2
Found fighter
20 completed with j: 3
Found fighter
21 completed with j: 7
Found fighter
22 completed with j: 7
Found fighter
23 completed with j: 4
Found fighter
24 completed with j: 4
Found fighter
25 completed with j: 6
Found fighter
26 completed with j: 15
Found f

Found fighter
208 completed with j: 5
Found fighter
209 completed with j: 3
Found fighter
210 completed with j: 3
Found fighter
211 completed with j: 6
Found fighter
212 completed with j: 3
Found fighter
213 completed with j: 3
Found fighter
214 completed with j: 3
Found fighter
215 completed with j: 11
Found fighter
216 completed with j: 14
Found fighter
217 completed with j: 12
Found fighter
218 completed with j: 3
Found fighter
219 completed with j: 5
Found fighter
220 completed with j: 3
Found fighter
221 completed with j: 7
Found fighter
222 completed with j: 4
Found fighter
223 completed with j: 3
Found fighter
224 completed with j: 5
Found fighter
225 completed with j: 7
Found fighter
226 completed with j: 3
Found fighter
227 completed with j: 2
Found fighter
228 completed with j: 3
Found fighter
229 completed with j: 4
Found fighter
230 completed with j: 2
Found fighter
231 completed with j: 8
Found fighter
232 completed with j: 9
Found fighter
233 completed with j: 7
Found fig

Found fighter
423 completed with j: 2
Found fighter
424 completed with j: 6
Found fighter
425 completed with j: 3
Found fighter
426 completed with j: 3
Found fighter
Error reading fighter  255 , try:  1
Found fighter
Error reading fighter  255 , try:  2
Found fighter
Error reading fighter  255 , try:  3
Found fighter
Error reading fighter  255 , try:  4
Found fighter
Error reading fighter  255 , try:  5
Found fighter
B max i:  256  and time: Thu Sep 24 15:50:00 2020
Found fighter
427 completed with j: 8
Found fighter
428 completed with j: 7
Found fighter
429 completed with j: 25
Found fighter
430 completed with j: 3
Found fighter
431 completed with j: 6
Found fighter
432 completed with j: 3
Found fighter
433 completed with j: 2
Found fighter
434 completed with j: 13
Found fighter
435 completed with j: 2
Found fighter
436 completed with j: 2
Found fighter
437 completed with j: 10
Found fighter
438 completed with j: 3
Found fighter
439 completed with j: 9
Found fighter
440 completed with

...

Found fighter
2603 completed with j: 5
Found fighter
2604 completed with j: 13
Found fighter
2605 completed with j: 2
Found fighter
2606 completed with j: 2
Found fighter
2607 completed with j: 8
Found fighter
2608 completed with j: 3
Found fighter
2609 completed with j: 5
Found fighter
2610 completed with j: 2
Found fighter
2611 completed with j: 4
Found fighter
2612 completed with j: 3
Found fighter
2613 completed with j: 2
Found fighter
2614 completed with j: 2
Found fighter
2615 completed with j: 4
Found fighter
2616 completed with j: 3
Found fighter
2617 completed with j: 2
Found fighter
2618 completed with j: 4
Found fighter
2619 completed with j: 5
Found fighter
2620 completed with j: 15
Found fighter
2621 completed with j: 2
Found fighter
2622 completed with j: 6
Found fighter
2623 completed with j: 3
Found fighter
2624 completed with j: 6
Found fighter
2625 completed with j: 13
Found fighter
2626 completed with j: 3
Found fighter
2627 completed with j: 3
Found fighter
2628 com

Found fighter
2803 completed with j: 6
Found fighter
2804 completed with j: 9
Found fighter
2805 completed with j: 3
Found fighter
2806 completed with j: 3
Found fighter
2807 completed with j: 4
Found fighter
2808 completed with j: 6
Found fighter
2809 completed with j: 3
Found fighter
2810 completed with j: 4
Found fighter
2811 completed with j: 5
Found fighter
2812 completed with j: 2
Found fighter
2813 completed with j: 3
Found fighter
2814 completed with j: 2
Found fighter
2815 completed with j: 11
Found fighter
2816 completed with j: 5
Found fighter
2817 completed with j: 2
Found fighter
2818 completed with j: 2
Found fighter
2819 completed with j: 3
Found fighter
2820 completed with j: 4
Found fighter
2821 completed with j: 3
Found fighter
2822 completed with j: 3
Found fighter
2823 completed with j: 10
Found fighter
2824 completed with j: 3
Found fighter
2825 completed with j: 5
Found fighter
2826 completed with j: 10
Found fighter
2827 completed with j: 21
Found fighter
2828 co

3012 completed with j: 12
Found fighter
3013 completed with j: 19
Found fighter
3014 completed with j: 13
Found fighter
3015 completed with j: 4
Found fighter
3016 completed with j: 3
Found fighter
3017 completed with j: 3
Found fighter
3018 completed with j: 3
Found fighter
3019 completed with j: 7
Found fighter
3020 completed with j: 21
Found fighter
3021 completed with j: 2
Found fighter
3022 completed with j: 4
Found fighter
3023 completed with j: 2
Found fighter
3024 completed with j: 5
Found fighter
3025 completed with j: 3
Found fighter
3026 completed with j: 3
Found fighter
3027 completed with j: 4
Found fighter
3028 completed with j: 14
Found fighter
3029 completed with j: 14
Found fighter
3030 completed with j: 5
Found fighter
3031 completed with j: 4
Found fighter
3032 completed with j: 10
Found fighter
3033 completed with j: 2
Found fighter
3034 completed with j: 9
Found fighter
3035 completed with j: 10
Found fighter
3036 completed with j: 29
Found fighter
3037 completed w

Found fighter
3213 completed with j: 4
Found fighter
3214 completed with j: 12
Found fighter
3215 completed with j: 3
Found fighter
3216 completed with j: 2
Found fighter
3217 completed with j: 2
Found fighter
3218 completed with j: 3
Found fighter
3219 completed with j: 18
Found fighter
3220 completed with j: 2
Found fighter
3221 completed with j: 6
Found fighter
3222 completed with j: 6
Found fighter
3223 completed with j: 3
Found fighter
3224 completed with j: 3
Found fighter
3225 completed with j: 3
Found fighter
3226 completed with j: 4
Found fighter
3227 completed with j: 14
Found fighter
3228 completed with j: 4
Found fighter
3229 completed with j: 3
Found fighter
3230 completed with j: 14
Found fighter
3231 completed with j: 3
Found fighter
3232 completed with j: 2
Found fighter
3233 completed with j: 4
Found fighter
3234 completed with j: 2
Found fighter
3235 completed with j: 5
Found fighter
3236 completed with j: 6
Found fighter
3237 completed with j: 3
Found fighter
3238 co

Found fighter
3397 completed with j: 5
Found fighter
3398 completed with j: 14
Found fighter
3399 completed with j: 2
Found fighter
3400 completed with j: 2
Found fighter
3401 completed with j: 4
Found fighter
3402 completed with j: 4
Found fighter
3403 completed with j: 6
Found fighter
3404 completed with j: 9
Found fighter
3405 completed with j: 2
Found fighter
3406 completed with j: 5
Found fighter
3407 completed with j: 4
Found fighter
3408 completed with j: 3
Found fighter
3409 completed with j: 5
Found fighter
3410 completed with j: 2
Found fighter
3411 completed with j: 18
Found fighter
3412 completed with j: 6
Found fighter
3413 completed with j: 7
Found fighter
3414 completed with j: 2
Found fighter
3415 completed with j: 34
Found fighter
3416 completed with j: 3
Found fighter
3417 completed with j: 4
Found fighter
3418 completed with j: 5
Found fighter
3419 completed with j: 3
Found fighter
3420 completed with j: 6
Found fighter
3421 completed with j: 8
Found fighter
3422 com

## Save Model

In [ ]:
import os
import subprocess
import datetime

In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [ ]:
# Optional: Save Dataframe
date = datetime.datetime.today().strftime('%Y-%m-%d')

In [ ]:
html_path = "".join([cfg.path_data_output,date,'_ufcstats_scraper.html'])

%notebook "D:/jasper/98_general_projects/03_UFC_Predictions/Backup/ufcstats_scraper.ipynb"  
_ = subprocess.call(f'jupyter nbconvert ufcstats_scraper.ipynb --output D:\\jasper\\98_general_projects\\03_UFC_Predictions/Backup/{date}_ufcstats_scraper.html"'); del _
print('\nFEIERABEND!!!')